In [5]:
from transformers import pipeline
import torch
import pandas as pd
import numpy as np

In [10]:
device = "mps" if torch.backends.mps.is_available() else "cuda:1" if torch.cuda.is_available() else "cpu"


model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path, device=device)

print(device)

mps


In [7]:
sentiment_task("I love you")

[{'label': 'positive', 'score': 0.7866936326026917}]

In [6]:
filtered_df_sport_category = pd.read_parquet(
    'filtered_sport_metadata(by_tags_and_title)_without_description_column.parquet')

In [ ]:
# Apply the sentiment analysis to each title and store the results in a new column

count = 0
def sentiment_analysis(title):
    global count
    if count % 1_000_000 == 0:
        print(count)
    count += 1

    return sentiment_task(title)[0]['label']

filtered_df_sport_category['title_sentiment'] = filtered_df_sport_category['title'].apply(lambda x: sentiment_analysis(x))

0
1000


KeyboardInterrupt: 

In [ ]:
print(filtered_df_sport_category.head())